# Datos de accidentes de tráfico en Madrid (2017-2018)

**Fuente**: [datos.madrid.es](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=7c2843010d9c3610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)

## Descripción

Datos en los ficheros:

- FECHA: formato dd/mm/aaaa
- RANGO HORARIO: rango horario en el que ocurrió el accidente.
- DIA SEMANA
- DISTRITO
- LUGAR ACCIDENTE
- Nº: número de la calle (sólo si tiene sentido).
- Nº PARTE: identificador del accidente. Si distintos registros tienen el mismo número parte, se trata del mismo accidente donde hay varios afectados.
- CPFA Granizo: condiciones ambientales que se dan en el momento del siniestro, granizo.
- CPFA Hielo: condiciones ambientales que se dan en el momento del siniestro, hielo.
- CPFA Lluvia: condiciones ambientales que se dan en el momento del siniestro, lluvia.
- CPFA Niebla: condiciones ambientales que se dan en el momento del siniestro, niebla.
- CPFA Seco: condiciones ambientales que se dan en el momento del siniestro, seco.
- CPFA Nieve: condiciones ambientales que se dan en el momento del siniestro, nieve.
- CPSV Mojada: circunstancias que afectan el estado de la vía en el momento de producirse el siniestro, mojada.
- CPSV Aceite: circunstancias que afectan el estado de la vía en el momento de producirse el siniestro, aceite.
- CPSV Barro: circunstancias que afectan el estado de la vía en el momento de producirse el siniestro, barro.
- CPSV Grava Suelta: circunstancias que afectan el estado de la vía en el momento de producirse el siniestro, grava suelta.
- CPSV Hielo: circunstancias que afectan el estado de la vía en el momento de producirse el siniestro, hielo.
- CPSV Seca Y Limpia: circunstancias que afectan el estado de la vía en el momento de producirse el siniestro, seca y limpia.
- Nº VICTIMAS
- TIPO ACCIDENTE: colisión doble, colisión múltiple, choque con objeto fijo, atropello, vuelco, caída motocicleta, caída ciclomotor, caída bicicleta, caída viajero bus.
- Tipo Vehiculo
- TIPO PERSONA: Conductor, peatón, testigo o viajero.
- SEXO: hombre, mujer o no asignado
- LESIVIDAD: muerto, herido grave, herido leve, ileso
- Tramo Edad

### Carga de datos

In [ ]:
import pandas as pd

# Se cargan los datos y se renombran todas las columnas
new_names = [
    'fecha', 'rango_horario', 'dia_semana', 'distrito',
    'lugar_accidente', 'num_calle', 'num_parte', 'fa_granizo',
    'fa_hielo', 'fa_lluvia', 'fa_niebla', 'fa_seco', 'fa_nieve',
    'sv_mojado', 'sv_aceite', 'sv_barro', 'sv_grava_suelta',
    'sv_hielo', 'sv_seca_limpia', 'num_victimas', 'tipo_accidente',
    'tipo_vehiculo', 'tipo_persona', 'sexo', 'lesividad', 'tramo_edad'
]

# Datos para 2017
df_accidents_2017 = pd.read_excel(
    'https://datos.madrid.es/egob/catalogo/300228-1-accidentes-trafico-detalle.xlsx',
    skiprows=0, names=new_names
)

# Datos para 2018
df_accidents_2018 = pd.read_excel(
    'https://datos.madrid.es/egob/catalogo/300228-0-accidentes-trafico-detalle.xlsx',
    skiprows=0, names=new_names
)

# Unión de datos 2017-2018
df_all_accidents = pd.concat([df_accidents_2017, df_accidents_2018], sort=False)

#### Víctimas por accidente

In [ ]:
s_num_victims_by_accident = df_all_accidents.groupby(['num_parte'])['num_victimas'].sum()

#### Accidentes por distrito y por año

In [ ]:
df_accidents_by_district = pd.pivot_table(
    df_all_accidents, index=['distrito'], columns=[df_all_accidents['fecha'].dt.year],
    aggfunc=pd.Series.nunique, values='num_parte'
)

# Se verifica que el resultado es correcto
if not df_accidents_by_district[2017].sum() == df_accidents_2017['num_parte'].nunique():
    raise Exception("Error para el año 2017")
if not df_accidents_by_district[2018].sum() == df_accidents_2018['num_parte'].nunique():
    raise Exception("Error para el año 2018")

#### Accidentes sin factores climáticos desfavorables

In [ ]:
df_best_weather = df_all_accidents[
    (df_all_accidents['sv_seca_limpia'] == 'SI') & (df_all_accidents['fa_seco'] == 'SI')
]

####  Accidentes ocurridos en días de semana

In [ ]:
weekdays = ['LUNES', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES']

df_accidents_on_weekdays = df_all_accidents[df_all_accidents['dia_semana'].isin(weekdays)]